In [1]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.5/206.5 kB 21.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.7 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=bf831070bdcb3777f45451254ce13f264a13c9cffcceadbdd80b108866a05a97
  Stored in directory: /root/.cache/pip/wheels/e7/f3/22/152153d6eb222ee7a56ff8617d80ee5207207a8c00a7aab794
Successfully built pathtools


In [4]:
!wandb --version

wandb, version 0.15.3


In [5]:
%%capture
!wget -nc https://github.com/DataTalksClub/mlops-zoomcamp/raw/main/cohorts/2023/02-experiment-tracking/homework-wandb/preprocess_data.py -P scripts_wb

In [7]:
!python scripts_wb/preprocess_data.py \
  --wandb_project 'mlops-zoomcamp' \
  --raw_data_path ./data \
  --dest_path ./output

wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice: 1
wandb: You chose 'Create a W&B account'
wandb: Create an account here: https://wandb.ai/authorize?signup=true
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Tracking run with wandb version 0.15.3
wandb: Run data is saved locally in /content/wandb/run-20230606_173857-e6akbceo
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run quiet-waterfall-1
wandb: ⭐️ View project at https://wandb.ai/vidya-ratan96/mlops-zoomcamp
wandb: 🚀 View run at https://wandb.ai/vidya-ratan96/mlops-zoomcamp/runs/e6akbceo
wandb: Adding directory to artifact (./output)... Done. 0.0s
wandb: Waiting for W&B process to finish... (success).
wandb: 🚀 View run quiet-waterfall-1 at: https://wandb.ai/vidya-ratan96/mlops-zoomcamp/runs/e6akbceo
wandb: Synced 5 W&B

In [8]:
from types import SimpleNamespace

config = SimpleNamespace(
    WANDB_PROJECT='mlops-zoomcamp',
    WANDB_ENTITY=None,
    ARTIFACT_NAME='vidya-ratan96/mlops-zoomcamp/NYC-Taxi:v0',
    
)

In [9]:
import os
import pickle

import wandb

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error


def load_pickle(filename: str):
    with open(filename, "rb") as f_in:
        return pickle.load(f_in)


def run_train(
    wandb_project: str,
    wandb_entity: str,
    data_artifact: str,
    max_depth: int,
    random_state: int,
):
    # Initialize a Weights & Biases run
    wandb.init(
        project=wandb_project,
        entity=wandb_entity,
        job_type="train",
        config={"max_depth": max_depth, "random_state": random_state},
    )

    # Fetch the preprocessed dataset from artifacts
    artifact = wandb.use_artifact(data_artifact, type="preprocessed_dataset")
    data_path = artifact.download()

    X_train, y_train = load_pickle(os.path.join(data_path, "train.pkl"))
    X_val, y_val = load_pickle(os.path.join(data_path, "val.pkl"))

    # Define the XGBoost Regressor Mode, train the model and perform prediction
    rf = RandomForestRegressor(max_depth=max_depth, random_state=random_state)
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_val)

    mse = mean_squared_error(y_val, y_pred, squared=False)
    # TODO: Log `mse` to Weights & Biases under the key `"MSE"`
    wandb.log({'MSE': mse})

    with open("regressor.pkl", "wb") as f:
        pickle.dump(rf, f)

    # TODO: Log `regressor.pkl` as an artifact of type `model`
    regressor_model = wandb.Artifact('regressor', type='model')
    regressor_model.add_file("regressor.pkl")
    wandb.log_artifact(regressor_model)

In [10]:
run_train(
    wandb_project=config.WANDB_PROJECT,
    wandb_entity=config.WANDB_ENTITY,
    data_artifact=config.ARTIFACT_NAME,
    max_depth=10,
    random_state=0,)

wandb: Currently logged in as: vidya-ratan96. Use `wandb login --relogin` to force relogin


wandb:   4 of 4 files downloaded.  


In [11]:
wandb.finish()

MSE,▁
MSE,2.45398


In [12]:
import os
import pickle
from functools import partial

import wandb

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error


def load_pickle(filename: str):
    with open(filename, "rb") as f_in:
        return pickle.load(f_in)


def run_train(data_artifact: str):
    wandb.init()
    config = wandb.config

    # Fetch the preprocessed dataset from artifacts
    artifact = wandb.use_artifact(data_artifact, type="preprocessed_dataset")
    data_path = artifact.download()

    X_train, y_train = load_pickle(os.path.join(data_path, "train.pkl"))
    X_val, y_val = load_pickle(os.path.join(data_path, "val.pkl"))

    # Define the XGBoost Regressor Mode, train the model and perform prediction
    # TODO: Pass the parameters n_estimators, min_samples_split, min_samples_leaf from `config` to `RandomForestRegressor`
    rf = RandomForestRegressor(max_depth=config.max_depth, random_state=0,
                               n_estimators=config.n_estimators,
                               min_samples_split=config.min_samples_split,
                               min_samples_leaf=config.min_samples_leaf)
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_val)

    mse = mean_squared_error(y_val, y_pred, squared=False)
    wandb.log({"MSE": mse})

    with open("regressor.pkl", "wb") as f:
        pickle.dump(rf, f)

    artifact = wandb.Artifact(f"{wandb.run.id}-model", type="model")
    artifact.add_file("regressor.pkl")
    wandb.log_artifact(artifact)
    wandb.finish()


SWEEP_CONFIG = {
    "method": "bayes",
    "metric": {"name": "MSE", "goal": "minimize"},
    "parameters": {
        "max_depth": {
            "distribution": "int_uniform",
            "min": 1,
            "max": 20,
        },
        "n_estimators": {
            "distribution": "int_uniform",
            "min": 10,
            "max": 50,
        },
        "min_samples_split": {
            "distribution": "int_uniform",
            "min": 2,
            "max": 10,
        },
        "min_samples_leaf": {
            "distribution": "int_uniform",
            "min": 1,
            "max": 4,
        },
    },
}


def run_sweep(wandb_project: str, wandb_entity: str, data_artifact: str, count: int):
    sweep_id = wandb.sweep(SWEEP_CONFIG, project=wandb_project, entity=wandb_entity)
    wandb.agent(sweep_id, partial(run_train, data_artifact=data_artifact), count=count)

In [13]:
run_sweep(wandb_project=config.WANDB_PROJECT,
           wandb_entity=config.WANDB_ENTITY, 
           data_artifact=config.ARTIFACT_NAME, 
           count=5)

Create sweep with ID: jklyf9zq
Sweep URL: https://wandb.ai/vidya-ratan96/mlops-zoomcamp/sweeps/jklyf9zq


wandb: Agent Starting Run: bczr8z7i with config:
wandb: 	max_depth: 10
wandb: 	min_samples_leaf: 1
wandb: 	min_samples_split: 2
wandb: 	n_estimators: 10


wandb:   4 of 4 files downloaded.  


MSE,▁
MSE,2.45912


wandb: Agent Starting Run: 9f8uun9h with config:
wandb: 	max_depth: 16
wandb: 	min_samples_leaf: 3
wandb: 	min_samples_split: 3
wandb: 	n_estimators: 24


...

wandb:   4 of 4 files downloaded.  


MSE,▁
MSE,2.44794


wandb: Agent Starting Run: 51xabrgt with config:
wandb: 	max_depth: 20
wandb: 	min_samples_leaf: 4
wandb: 	min_samples_split: 2
wandb: 	n_estimators: 29


wandb:   4 of 4 files downloaded.  


MSE,▁
MSE,2.45172
